In [ ]:
import rus_ner

rus_ner.slurp_NE5_annotated_data('/data/NER/VectorX/NE5_train', 'train')
rus_ner.slurp_NE5_annotated_data('/data/NER/VectorX/NE5_test', 'test')
train = rus_ner.get_supervised_data('train')
test = rus_ner.get_supervised_data('train')

Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_train/82207990.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_train/82157720.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_train/82229530.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_train/82218580.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_train/82178650.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_train/82217490.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_test/84272795.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_test/84296221.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_test/84261598.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_test/84261780.txt
Exception "Entity is in two sentences" for file /data/NER/VectorX/NE5_test/84303229.txt
Exception "Entity is in tw

In [2]:
ud = rus_ner.get_unsupervised_data()

In [ ]:
ud

In [ ]:
import sqlite3
from pathlib import Path
import csv
from typing import List, Tuple, Callable, Iterable
from nltk import sent_tokenize, word_tokenize
import pandas as pd

NE5_dataset = Path('/data/NER/VectorX/NE5_train')
# NE5_dataset = Path('/data/NER/Academic Datasets/NE5/Collection5')

In [ ]:
def get_spans(text, sentences):
    sents_span = []

    last_index = -1
    for s in sentences:
        i = text[last_index+1:].index(s)
        span_start = last_index + 1 + i
        span_end = last_index + 1 + i + len(s)

        sents_span.append((s, span_start, span_end))
        last_index = span_end

    return sents_span


# def get_spans(text, tokenizer, subs_map=None):
#     if subs_map is None:
#         subs_map = {}
        
#     wi = 0
#     words = tokenizer(text)
#     word_origs = [subs_map.get(w, w) for w in words]
#     words_span = []

#     i = 0
#     while i < len(text):
#         if text[i:].startswith(word_origs[wi]):
#             words_span.append((words[wi], i, i+len(word_origs[wi])))
#             i += len(word_origs[wi])
#             wi += 1
#             continue
#         i += 1

#     return words_span


def split_sentences_preserve_tags(text : str, 
                                  tags_with_spans: Iterable[Tuple[str, int, int, str]], 
                                  sentence_tokenizer: Callable[[str], Iterable[str]]):
    tags_with_spans = sorted(tags_with_spans, key=lambda x: x[1])  # excess
    sents = sentence_tokenizer(text)
#     print(text)
#     sents_span = get_spans(text, sentence_tokenizer)
    sents_span = get_spans(text, sents)

    per_sentence_tags = []
    for orig_s, s_start, s_end in sents_span:
        sent_tags = []
        s = orig_s.strip()
        offset = orig_s.index(s)
        s_start = s_start - offset
        s_end = s_end - offset
        
        for tag, t_start, t_end, value in tags_with_spans:
            if s_start <= t_start <= s_end:
                assert s_start <= t_end <= s_end, 'Entity is in two sentences'
                assert s[t_start - s_start: t_end - s_start] == value, 'Tag value does not equal to sent spans'
                sent_tags.append((tag, t_start - s_start, t_end - s_start, value))
        per_sentence_tags.append((s, sent_tags))
    return per_sentence_tags


def spans_to_bio(word_spans: List[Tuple[str, int, int]], tags: List[Tuple[int, int, str]]):
    '''supposed that tags do not overlap'''
    tags = sorted(tags, key=lambda x: x[0])
    for t1, t2 in zip(tags[:-1], tags[1:]):
        assert t1[1] <= t2[0], 'Tags must not overlap'
    
    bio = []
    for t_start, t_end, tag in tags:
        last_tag = 'O'    
        for w, w_start, w_end in word_spans:
            if t_start <= w_start < t_end or t_start < w_end < t_end:
                prefix = 'I-' if last_tag == tag else 'B-'
                bio.append((w, prefix + tag))
                last_tag = tag
            else:
                bio.append((w, 'O'))
                last_tag = 'O'
    return bio


In [ ]:
conn = sqlite3.connect(':memory:')

c = conn.cursor()

c.execute('''CREATE TABLE sentences (article_id INT, 
                                     value TEXT)''')
c.execute('''CREATE TABLE tags (sentence_id INT, 
                                start_index INT, 
                                end_index INT, 
                                tag TEXT)''')
conn.commit()

In [ ]:
all_texts = []
all_annotations = []
for ann_fn in NE5_dataset.glob('*.ann'):
    c = conn.cursor()
#     ann_fn = Path('/data/NER/VectorX/NE5_train/82180290.ann')
    article_id = ann_fn.name[:-len('.ann')]
    text_fn = Path(str(ann_fn)[:-len('.ann')] + '.txt')
    
    text = text_fn.read_text()
    all_texts.append({'article_id': article_id, 'text': text})
    
    annotations = []
    with ann_fn.open() as f:
        cr = csv.reader(f, delimiter='\t', quotechar='|')
        for row in cr:
            assert len(row) == 3, row
            _, tag_info, tag_value = row
            tag, start_index, end_index = tag_info.split()
            annotations.append((tag, int(start_index), int(end_index), tag_value))

    st = split_sentences_preserve_tags(text, annotations, sent_tokenize)
    
    
    for s, stags in st:
        r = c.execute('''INSERT INTO sentences(article_id, value) VALUES(?,?)''', (article_id, s))
        s_id = r.lastrowid
        for st, ss, se, sv in stags:
            c.execute('INSERT INTO tags(sentence_id, start_index, end_index, tag) VALUES (?, ?, ?, ?)', (s_id, ss, se, st))

    conn.commit()

In [ ]:
r = c.execute('SELECT sentences.oid, sentences.article_id, sentences.value, tags.start_index, tags.end_index, tags.tag FROM sentences LEFT OUTER JOIN tags ON sentences.oid = tags.sentence_id')
d = r.fetchall()

data = pd.DataFrame(d, columns='sentences.oid sentences.article_id sentences.value tags.start_index tags.end_index tags.tag'.split())

In [ ]:
sents = []
for sid, d in data.groupby('sentences.oid'):
    s = d['sentences.value'].iloc[0]
    tags = []
    for _, row in d.iterrows():
        if not pd.isna(row['tags.start_index']):
            tags.append((int(row['tags.start_index']), int(row['tags.end_index']), row['tags.tag']))
    sents.append((s, tags))

In [ ]:
text, tags = sents[1]
nltk_subs_map = {"''": '"', '``': '"'}

word_spans = get_spans(text, word_tokenize, nltk_subs_map)

In [ ]:
tags

In [ ]:
word_spans